In [5]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_ccba = pd.read_csv("./訓練資料集_first/public_train_x_ccba_full_hashed.csv")
df_cdtx = pd.read_csv("./訓練資料集_first/public_train_x_cdtx0001_full_hashed.csv")
df_custinfo = pd.read_csv("./訓練資料集_first/public_train_x_custinfo_full_hashed.csv")
df_dp = pd.read_csv("./訓練資料集_first/public_train_x_dp_full_hashed.csv")
df_remit1 = pd.read_csv("./訓練資料集_first/public_train_x_remit1_full_hashed.csv")
df_alert_date = pd.read_csv("./訓練資料集_first/train_x_alert_date.csv")
df_public_alert_date = pd.read_csv("./訓練資料集_first/public_x_alert_date.csv")


In [4]:
df_answer = pd.read_csv("./訓練資料集_first/train_y_answer.csv")
df_answer.head()

,alert_key,sar_flag
0,171189,0
1,171202,0
2,171599,0
3,171737,0
4,171142,0


# 1. Merge alert key data into cust info data

## key map  : 
(alert_date)**alert key** -> (custinfo) **alert key** (many to one)

In [24]:
data = df_alert_date.merge(df_custinfo, how= 'left', on = 'alert_key')
data

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE
0,171189,0,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17.0,375576.0,4
1,171202,0,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12.0,2717416.0,2
2,171599,0,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12.0,326517.0,4
3,171737,0,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14.0,1014759.0,4
4,171142,0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3
...,...,...,...,...,...,...,...
23901,352132,364,c49b33d1fde790ec03584672903f296e486615adfdc989...,1,19.0,3218731.0,3
23902,352125,364,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c2...,3,19.0,928963.0,3
23903,352080,364,1a93961c8fa830a1f32f5316b44f4964d65397f0311b11...,2,9.0,69080.0,6
23904,352075,364,9586b80d3ad1d74c0a1efe792cae2ecd073243fb01ed33...,3,3.0,262604.0,6


## cumulative alert times

In [12]:
alert_times_dict = {}
alert_times_list = []
for i, row in data.iterrows():
    if alert_times_dict.__contains__(row.cust_id):
        alert_times_list.append(alert_times_dict[row.cust_id])
        alert_times_dict[row.cust_id] += 1
    else :
        alert_times_list.append(0)
        alert_times_dict[row.cust_id] = 1
data['alert_times'] = alert_times_list
data

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,alert_times
0,171189,0,972ee157e63316e8a50dd489bc93730a3ee8a8959d5c6b...,1,17.0,375576.0,4,0
1,171202,0,a10ab33f90926fb18d7bb5e78034d7f04a1fbed95b7951...,3,12.0,2717416.0,2,0
2,171599,0,3433ecc068ed1c9e2f5037cab5d42d7b901e9bd624c1fb...,1,12.0,326517.0,4,0
3,171737,0,a0861608615a9365d90f4ba0a813c0ea0471987f925c8b...,3,14.0,1014759.0,4,0
4,171142,0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,3,12.0,241719.0,3,0
...,...,...,...,...,...,...,...,...
23901,352132,364,c49b33d1fde790ec03584672903f296e486615adfdc989...,1,19.0,3218731.0,3,2
23902,352125,364,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c2...,3,19.0,928963.0,3,1
23903,352080,364,1a93961c8fa830a1f32f5316b44f4964d65397f0311b11...,2,9.0,69080.0,6,6
23904,352075,364,9586b80d3ad1d74c0a1efe792cae2ecd073243fb01ed33...,3,3.0,262604.0,6,4


# 2. Merge ccba (credit card balance)

## key map  : 
1. (custinfo) **cust id** -> (ccba) **cust id** (one to many)
2. (alert_date) **date** -> (ccba) **byymm** (some of scale) 

In [13]:
data = data.merge(df_ccba, how = 'outer', left_on=['cust_id', 'date'], right_on=['cust_id', 'byymm'])
data['date'] = data[['date', 'byymm']].apply(lambda x : x[0] if pd.notna(x[0]) else x[1], axis = 1)
data.drop('byymm', axis = 1,inplace=True)
data = data.sort_values(['cust_id','alert_key']).reset_index(drop= True) # make cust info in the first of cust id to fill cust info
data

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,alert_times,lupay,cycam,usgam,clamt,csamt,inamt,cucsm,cucah
0,195913.0,53.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,NaN,NaN,NaN,NaN,NaN,3920.0,134614.0,32316.0,0.0,0.0,3148.0,16576.0,0.0
2,NaN,30.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,NaN,NaN,NaN,NaN,NaN,16576.0,134614.0,22280.0,0.0,0.0,3148.0,8653.0,0.0
3,NaN,61.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,NaN,NaN,NaN,NaN,NaN,8653.0,134614.0,14771.0,0.0,0.0,3148.0,5045.0,0.0
4,NaN,91.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,NaN,NaN,NaN,NaN,NaN,5045.0,134614.0,10954.0,0.0,0.0,3148.0,4334.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82534,NaN,334.0,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,NaN,NaN,NaN,NaN,NaN,19766.0,781570.0,26360.0,0.0,0.0,1142.0,2568.0,0.0
82535,NaN,365.0,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,NaN,NaN,NaN,NaN,NaN,2591.0,781570.0,23903.0,0.0,0.0,5199.0,5199.0,0.0
82536,350878.0,363.0,fff322436b556032e07690f12456d9eff3588c269d6210...,2.0,19.0,591218.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82537,351619.0,364.0,fff322436b556032e07690f12456d9eff3588c269d6210...,2.0,19.0,624147.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# fill value and resorted by date
# alert times need to fill by sorted data by date
data.loc[:, data.columns!='alert_times'] = data.loc[:, data.columns!='alert_times'].fillna(method='ffill')
data.loc[:, data.columns!='alert_times'] = data.loc[:, data.columns!='alert_times'].fillna(method='bfill')
data = data.sort_values(['cust_id', 'date'])
data.fillna(method='ffill', inplace=True)
data.fillna(0, inplace=True)
data

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,alert_times,lupay,cycam,usgam,clamt,csamt,inamt,cucsm,cucah
1,195913.0,0.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,3920.0,134614.0,32316.0,0.0,0.0,3148.0,16576.0,0.0
2,195913.0,30.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,16576.0,134614.0,22280.0,0.0,0.0,3148.0,8653.0,0.0
0,195913.0,53.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,16576.0,134614.0,22280.0,0.0,0.0,3148.0,8653.0,0.0
3,195913.0,61.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,8653.0,134614.0,14771.0,0.0,0.0,3148.0,5045.0,0.0
4,195913.0,91.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,5045.0,134614.0,10954.0,0.0,0.0,3148.0,4334.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82534,303894.0,334.0,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,1.0,14.0,458455.0,5.0,0.0,19766.0,781570.0,26360.0,0.0,0.0,1142.0,2568.0,0.0
82535,303894.0,365.0,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,1.0,14.0,458455.0,5.0,0.0,2591.0,781570.0,23903.0,0.0,0.0,5199.0,5199.0,0.0
82536,350878.0,363.0,fff322436b556032e07690f12456d9eff3588c269d6210...,2.0,19.0,591218.0,3.0,0.0,2591.0,781570.0,23903.0,0.0,0.0,5199.0,5199.0,0.0
82537,351619.0,364.0,fff322436b556032e07690f12456d9eff3588c269d6210...,2.0,19.0,624147.0,3.0,1.0,2591.0,781570.0,23903.0,0.0,0.0,5199.0,5199.0,0.0


In [22]:
data.head(50)

,alert_key,date,cust_id,risk_rank,occupation_code,total_asset,AGE,alert_times,lupay,cycam,usgam,clamt,csamt,inamt,cucsm,cucah
1,195913.0,0.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,3920.0,134614.0,32316.0,0.0,0.0,3148.0,16576.0,0.0
2,195913.0,30.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,16576.0,134614.0,22280.0,0.0,0.0,3148.0,8653.0,0.0
0,195913.0,53.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,16576.0,134614.0,22280.0,0.0,0.0,3148.0,8653.0,0.0
3,195913.0,61.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,8653.0,134614.0,14771.0,0.0,0.0,3148.0,5045.0,0.0
4,195913.0,91.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,5045.0,134614.0,10954.0,0.0,0.0,3148.0,4334.0,0.0
5,195913.0,122.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,4334.0,134614.0,7544.0,0.0,0.0,3148.0,4396.0,0.0
6,195913.0,153.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,4396.0,134614.0,4406.0,0.0,0.0,3148.0,4183.0,0.0
7,195913.0,183.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,4183.0,134614.0,46847.0,0.0,0.0,3157.0,10341.0,0.0
8,195913.0,214.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,7077.0,134614.0,40231.0,0.0,0.0,3148.0,9911.0,0.0
9,195913.0,244.0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1.0,17.0,32758.0,3.0,0.0,7480.0,134614.0,34726.0,0.0,0.0,3148.0,5671.0,0.0


# 3. Merge cdtx

In [28]:
df_cdtx[['country', 'cur_type']].value_counts()

country  cur_type
130      47          758908
134      46           72612
61       47           31410
134      30           25675
135      46           13601
                      ...  
19       24               1
94       37               1
         1                1
92       46               1
23       9                1
Length: 397, dtype: int64

# Merge with answer

In [ ]:
data = data.merge(df_answer, how= 'left', on = 'alert_key')